In [1]:
import mysql.connector as sql
from urllib.parse import quote
import os, warnings
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv()
warnings.simplefilter(action='ignore', category=FutureWarning)

engine = create_engine(f"mysql://{os.getenv('MYSQL_USER')}:%s@{os.getenv('MYSQL_HOST')}/DataAnalysis" % quote(os.getenv('MYSQL_PASSWORD'))) 

In [2]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine
from llama_index import SQLDatabase
warnings.simplefilter(action='ignore', category=FutureWarning)
sql_database = SQLDatabase(engine, sample_rows_in_table_info=2)
list(sql_database._all_tables)

C:\Users\kyle\AppData\Roaming\Python\Python311\site-packages\llama_index\utilities\sql_wrapper.py:110: SAWarning: Did not recognize type 'point' of column 'ListingCoordinates'
  self._metadata.reflect(
C:\Users\kyle\AppData\Roaming\Python\Python311\site-packages\llama_index\utilities\sql_wrapper.py:110: SAWarning: Did not recognize type 'point' of column 'CollegeCoordinates'
  self._metadata.reflect(
C:\Users\kyle\AppData\Roaming\Python\Python311\site-packages\llama_index\utilities\sql_wrapper.py:110: SAWarning: Did not recognize type 'point' of column 'SchoolCoordinates'
  self._metadata.reflect(
C:\Users\kyle\AppData\Roaming\Python\Python311\site-packages\llama_index\utilities\sql_wrapper.py:110: SAWarning: Did not recognize type 'point' of column 'UniversityCoordinates'
  self._metadata.reflect(
C:\Users\kyle\AppData\Roaming\Python\Python311\site-packages\llama_index\utilities\sql_wrapper.py:110: SAWarning: Did not recognize type 'point' of column 'BusinessCoordinates'
  self._metad

['AirbnbData',
 'ZillowListingsSchools',
 'CollegesData',
 'SchoolData',
 'RemaxListingsAssociations',
 'MortgageData',
 'CitiesData',
 'AirbnbDataAssociations',
 'UniversitiesData',
 'RemaxListingsWalkscore',
 'RemaxListingsAmeneties',
 'YelpData',
 'ZillowListingsAssociations',
 'RemaxListingsSchools',
 'RemaxListings',
 'ZillowListingsAmeneties',
 'ZillowListingsWalkscore',
 'ZillowListings',
 'RemaxListingsUniversities',
 'ZillowListingsColleges',
 'RemaxListingsColleges',
 'RemaxListingsAirbnb',
 'ZillowListingsAirbnb',
 'ZillowListingsUniversities',
 'YelpBusinessData']

In [6]:
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index import ServiceContext

MODEL_PATH = r"D:\Development Projects\Llam-Interpreter\llama-2-7b-chat.Q4_K_M.gguf"
llm = LlamaCPP(
    model_path=MODEL_PATH,
    temperature=0.1,
    max_new_tokens=256,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": 20},  # set to at least 1 to use GPU
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt
)

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

service_context = ServiceContext.from_defaults(
  llm=llm, chunk_size=1024, embed_model="local", callback_manager=callback_manager
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [7]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    service_context=service_context
)

In [8]:
table_details = {
    'CitiesData': 'Stores List of cities in Ontario, Canada and related data',
    'RemaxListings': 'Stores Property listing data in Ontario, Canada from Remax.ca',
    'RemaxListingsAssociations': 'Stores Property listings pricing data in timeseries form for all the real estate listings stored in RemaxListings table',
    'AirbnbDataAssociations': 'Stores Airbnb pricing data in timeseries form for all the real estate listings stored in AirbnbData table', 
    'RemaxListingsColleges': 'Stores Number of colleges stored in CollegeData table, near all real estate listings collected from Remax.ca and stored in RemaxListings table',
    'ZillowListingsAirbnb': 'Stores Number of Airbnbs, near all real estate listings collected from Zillow.ca and stored in ZillowListings table',
    'RemaxListingsWalkscore': 'Stores Walkscore for all real estate listings collected from Remax.ca and stored in RemaxListings table',
    'ZillowListings': 'Stores Property listing data in Ontario, Canada from Zillow.ca',
    'ZillowListingsWalkscore': 'Stores Walkscore for all real estate listings in Ontario, Canada collected from Zillow.ca and stored in ZillowListings table',
    'CollegesData': 'Stores List of colleges in Ontario, Canada',
    'YelpData': 'Stores List of business in Ontario, Canada and information about them collected from Yelp',
    'RemaxListingsAmeneties': 'Stores Number of amenties stored in YelpData table, near all real estate listings collected from Remax.ca and stored in RemaxListings table',
    'ZillowListingsSchools': 'Stores Number of schools near all real estate listings collected from Zillow.ca and stored in ZillowListings table',
    'RemaxListingsUniversities': 'Stores Number of universities near all real estate listings collected from Remax.ca and stored in RemaxListings table',
    'UniversitiesData': 'Stores List of universities in Ontario, Canada',
    'RemaxListingsSchools': 'Stores Number of schools near all real estate listings collected from Remax.ca and stored in RemaxListings table',
    'ZillowListingsUniversities': 'Stores Number of universities near all real estate listings collected from Zillow.ca and stored in ZillowListings table',
    'YelpBusinessData': 'Stores Meta data related to Yelp business data stored in YelpData table',
    'RemaxListingsAirbnb': 'Stores Number of Airbnbs, near all real estate listings collected from Remax.ca and stored in RemaxListings table',
    'ZillowListingsColleges': 'Stores Number of colleges stored in CollegeData table, near all real estate listings collected from Zillow.ca and stored in ZillowListings table',
    'SchoolData': 'Stores List of schools in ontario, Canada',
    'MortgageData': 'Stores House mortgage rates data from different lenders in timeseries form.',
    'ZillowListingsAssociations': 'Stores Property listings pricing data in timeseries form for all the real estate listings stored in ZillowListings table',
    'ZillowListingsAmeneties': 'Stores Number of amenties stored in YelpData table, near all real estate listings collected from Zillow.ca and stored in ZillowListings table',
    'AirbnbData': 'Stores List of Airbnbs in Ontario, Canada'
}

In [9]:
from llama_index.objects import ObjectIndex
from llama_index.objects import SQLTableNodeMapping, SQLTableSchema
import pandas as pd

tables = list(sql_database._all_tables)
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = []
for table in tables:
    table_schema_objs.append((SQLTableSchema(table_name = table, context_str = table_details[table])))

In [10]:
table_schema_objs

[SQLTableSchema(table_name='MortgageData', context_str='Stores House mortgage rates data from different lenders in timeseries form.'),
 SQLTableSchema(table_name='UniversitiesData', context_str='Stores List of universities in Ontario, Canada'),
 SQLTableSchema(table_name='ZillowListingsColleges', context_str='Stores Number of colleges stored in CollegeData table, near all real estate listings collected from Zillow.ca and stored in ZillowListings table'),
 SQLTableSchema(table_name='CitiesData', context_str='Stores List of cities in Ontario, Canada and related data'),
 SQLTableSchema(table_name='SchoolData', context_str='Stores List of schools in ontario, Canada'),
 SQLTableSchema(table_name='RemaxListingsUniversities', context_str='Stores Number of universities near all real estate listings collected from Remax.ca and stored in RemaxListings table'),
 SQLTableSchema(table_name='RemaxListingsSchools', context_str='Stores Number of schools near all real estate listings collected from Rem

In [11]:
from llama_index.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index import VectorStoreIndex


obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
    service_context=service_context
)

C:\Users\kyle\AppData\Roaming\Python\Python311\site-packages\llama_index\utilities\sql_wrapper.py:158: SAWarning: Did not recognize type 'point' of column 'UniversityCoordinates'
  for column in self._inspector.get_columns(table_name):
C:\Users\kyle\AppData\Roaming\Python\Python311\site-packages\llama_index\utilities\sql_wrapper.py:158: SAWarning: Did not recognize type 'point' of column 'SchoolCoordinates'
  for column in self._inspector.get_columns(table_name):
C:\Users\kyle\AppData\Roaming\Python\Python311\site-packages\llama_index\utilities\sql_wrapper.py:158: SAWarning: Did not recognize type 'point' of column 'ListingCoordinates'
  for column in self._inspector.get_columns(table_name):
C:\Users\kyle\AppData\Roaming\Python\Python311\site-packages\llama_index\utilities\sql_wrapper.py:158: SAWarning: Did not recognize type 'point' of column 'CollegeCoordinates'
  for column in self._inspector.get_columns(table_name):
C:\Users\kyle\AppData\Roaming\Python\Python311\site-packages\llama

**********
Trace: index_construction
    |_CBEventType.EMBEDDING ->  0.582854 seconds
    |_CBEventType.EMBEDDING ->  0.501073 seconds
    |_CBEventType.EMBEDDING ->  0.199294 seconds
**********


In [12]:
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=3), service_context=service_context
)

In [13]:
response = query_engine.query('Can you show me condo real estate listings in London, Ontario?')
print(response.response)